# Data acquisition

In [2]:
%pip install tensorflow

     -------------------------------------- 266.3/266.3 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 23.2/23.2 MB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.7 MB/s eta 0:00:00
     -------------------------------------- 895.9/895.9 kB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 7.8 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.3 MB/s eta 0:00:00
     -------------------------------------- 439.2/439.2 kB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 7.7 MB/s eta 0:00:00
     ------------------------------------- 177.2/177.2 kB


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tensorflow as tf

# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Vectorize the dataset
train_images = train_images.reshape((50000, 3072))
test_images = test_images.reshape((10000, 3072))

# Normalize the dataset
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# Compute the mean and standard deviation of the dataset
mean = train_images.mean(axis=0)
std = train_images.std(axis=0)

# Normalize the dataset using the mean and standard deviation
train_images = (train_images - mean) / std
test_images = (test_images - mean) / std


170498071/170498071 [==============================] - 21s 0us/step


# Implementation of MLP using NumPy

In [4]:
import numpy as np

class MLP:
    def __init__(self, activation, num_hidden_layers, hidden_units):
        self.activation = activation
        self.num_hidden_layers = num_hidden_layers
        self.hidden_units = hidden_units
        self.weights = []
        self.biases = []
        self.initialize_weights_and_biases()

    def initialize_weights_and_biases(self):
        input_dim = 3072
        output_dim = 10
        dims = [input_dim] + self.hidden_units + [output_dim]
        for i in range(len(dims) - 1):
            w = np.random.randn(dims[i], dims[i+1]) / np.sqrt(dims[i])
            b = np.zeros((1, dims[i+1]))
            self.weights.append(w)
            self.biases.append(b)

    def forward(self, X):
        A = X
        for i in range(self.num_hidden_layers):
            Z = np.dot(A, self.weights[i]) + self.biases[i]
            A = self.activation(Z)
        Z = np.dot(A, self.weights[-1]) + self.biases[-1]
        Y_hat = softmax(Z)
        return Y_hat

    def backward(self, X, Y, Y_hat, lr):
        m = X.shape[0]
        delta = Y_hat - Y
        for i in range(self.num_hidden_layers, 0, -1):
            dW = np.dot(self.activation(self.zs[i-1]).T, delta) / m
            db = np.sum(delta, axis=0, keepdims=True) / m
            delta = np.dot(delta, self.weights[i].T) * self.activation(self.zs[i-1], derivative=True)
            self.weights[i-1] -= lr * dW
            self.biases[i-1] -= lr * db
        dW = np.dot(X.T, delta) / m
        db = np.sum(delta, axis=0, keepdims=True) / m
        self.weights[0] -= lr * dW
        self.biases[0] -= lr * db

    def fit(self, X, y, lr, num_iterations, batch_size):
        num_examples = X.shape[0]
        num_batches = num_examples // batch_size
        for i in range(num_iterations):
            indices = np.random.permutation(num_examples)
            for j in range(num_batches):
                start_idx = j * batch_size
                end_idx = start_idx + batch_size
                batch_indices = indices[start_idx:end_idx]
                X_batch = X[batch_indices]
                y_batch = y[batch_indices]
                Y_hat = self.forward(X_batch)
                self.backward(X_batch, y_batch, Y_hat, lr)

    def predict(self, X):
        Y_hat = self.forward(X)
        return np.argmax(Y_hat, axis=1)

def softmax(Z):
    expZ = np.exp(Z)
    return expZ / np.sum(expZ, axis=1, keepdims=True)
